In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 13.18 GB
Used RAM: 1.97 GB
Percentage Usage Of RAM: 14.8%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 17.14 GB
Used Disk: 7.85 GB
Percentage Usage Of Disk: 31.4%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from wrangle import prepare
from nnet_regressor import Regression

[nltk_data] Downloading package vader_lexicon to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
2023-10-25 13:34:12.986605: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# get the data
crime = pd.read_csv("crime.csv")

In [5]:
# prepare the data for machine learning
crime = prepare(df=crime)

Data Wrangling:
> Converting Timestamps
> Computing Crime Rate
> Finding Days With No Crime
> Extracting Time Features
> Getting Economic Data
> Computing Weekly Crime Rate
> Computing Previous Weeks Of Crime
6.64 Seconds


In [6]:
# get the testing data
y = crime[["Crimes"]]
X = crime.drop(columns="Crimes")
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [7]:
# build the model
print("\n---- Crime Regression Analysis ----\n")
model = Regression(
    name="Tensorflow Without Feature Engineering", 
    path=None,
    rename=False, 
    deep=True,
    time=False, 
    text=False,
    binary=True, 
    imputation=False, 
    variance=True,
    scale=True,
    atwood=False,
    binning=False,
    reciprocal=False, 
    interaction=False, 
    selection=False,
    tune=False,
    plots=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.explore(crime)
    model.validate(X, y)  # build the machine learning pipeline
    predictions = model.predict(testX)
    print("\nModel Performance:")
    print(f"R2: {model.r2}")
    print(f"RMSE: {model.rmse}")
    print(f"In Control: {model.in_control}")


---- Crime Regression Analysis ----

Visualizing The Data:
> Plotting Correlations
> Crimes vs. Crimes(t-1)
> Crimes vs. Crimes(t-2)
> Crimes vs. Crimes(t-3)
> Crimes vs. Crimes(t-4)
> NASDAQ vs. GDP
> NASDAQ vs. GDI
> GDP vs. GDI
> Crimes(t-1) vs. Crimes(t-2)
> Crimes(t-1) vs. Crimes(t-3)
> Crimes(t-1) vs. Crimes(t-4)
> Crimes(t-2) vs. Crimes(t-3)
> Crimes(t-2) vs. Crimes(t-4)
> Crimes(t-3) vs. Crimes(t-4)
> Plotting Crimes
> Plotting NASDAQ
> Plotting Unemployment
> Plotting CPI
> Plotting PPI
> Plotting GDP
> Plotting GDI
> Plotting Federal_Funds_Rate
> Plotting Crimes(t-1)
> Plotting Crimes(t-2)
> Plotting Crimes(t-3)
> Plotting Crimes(t-4)
> Plotting Days
> Quarter vs. Days
> Month vs. Days
> Crimes vs. Dc_Dist
> Crimes vs. Year
> Crimes vs. Quarter
> Crimes vs. Month
> Crimes vs. Week
> Crimes vs. Days
> NASDAQ vs. Year
> NASDAQ vs. Week
> NASDAQ vs. Days
> Unemployment vs. Year
> Unemployment vs. Week
> Unemployment vs. Days
> CPI vs. Year
> CPI vs. Days
> PPI vs. Year
> GDP vs

In [8]:
# model diagnostics
print("Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print(" ")
print("Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Indicators:
1. Federal_Funds_Rate
2. GDI
3. GDP
4. PPI
5. CPI
6. Unemployment
7. Week_7
8. Week_9
9. NASDAQ
10. Week_8
 
Feature Drift:
1. Month_1
2. Quarter_1
3. Crimes
4. Crimes(t-4)
5. Crimes(t-3)
6. Crimes(t-2)
7. Crimes(t-1)
8. Year_2006
9. Year_2013
10. Year_2012


In [9]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 33.18211160192285
R2: 0.8492493815569798


In [10]:
# save the machine learning pipeline
model.dump()

In [11]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The Updated Data
> Transforming Categorical Features
> Removing Constant Features
> Scaling Features
> Training Neural Network



KeyboardInterrupt

